# Scrape Text from Government Websites

In [53]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment
from time import sleep

In [54]:
# specify the url
domain = "https://www.canada.ca"
root = "https://www.canada.ca/en/employment-social-development.html"
queue = [root]; # FIFO (FBS exploration)
visited = [] # Keeps track on what page have been visited
page_texts = [] # content scapped

In [55]:
EXTEND_TO_OUTSIDE_DOMAINS = False # Will scrape other domain content if a link referes to them
NUM_PAGES_TO_VISIT = 10000
PRINT_STEPS = 500
URL_MUST_CONTAIN = "employment-social-development"

In [56]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]',"nav","footer","form","label","details","summary"]:
        return False
    elif isinstance(element, Comment):
        return False
    else:
        return True

In [57]:
num_visited = 0
num_errors = 0
while(len(queue) > 0 and num_visited < NUM_PAGES_TO_VISIT):
    try:
        
        if URL_MUST_CONTAIN is not None:
            if URL_MUST_CONTAIN not in queue[0]:
                del queue[0]
                continue
                
        if not EXTEND_TO_OUTSIDE_DOMAINS:
            if domain not in queue[0]:
                del queue[0]
                continue
            
        if(queue[0] in visited): 
            #print("Page ({}) already visited".format(queue[0]))
            del queue[0]
            continue
            
        page = requests.get(queue[0])
        soup = BeautifulSoup(page.text, 'html.parser')
        texts = soup.find_all('h1')[0].find_all_next(text=True)#findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        page_text = " ".join([t.strip() for t in visible_texts if len(t.strip().split())>5]);

        page_texts.append(page_text)
        visited.append(queue[0])
        
        for link in soup("a"):
            if link.has_key('href'):
                if '.html' in link['href']:
                    if (".ca" in link['href'] or ".com" in link['href'] or "http:" in link['href'] or "www." in link['href']) and link['href'] not in queue:
                        queue.append(link['href'])    
                    elif link['href'].startswith("/") and domain+link['href'] not in queue:
                        queue.append(domain+link['href'])
                    else:
                        True
                        #print(link['href'] + " was not added to the queue (probably under a different domain):s")
                                     
        num_visited+=1
        
        if((num_visited)%PRINT_STEPS == 0):
            print("Page {} complete({})".format(num_visited,queue[0]))
        
        num_errors = 0
        del queue[0]
        #sleep(0.5)

    except Exception as e:
        #print("error for page {}: ".format(num_visited) + str(e))
        
        num_errors += 1
        if num_errors > 5:
            del queue[0]
            
        sleep(2)
        continue;

/usr/local/lib/python3.6/site-packages/bs4/element.py:1577: UserWarning: has_key is deprecated. Use has_attr("href") instead.
  key))


Page 500 complete(https://www.canada.ca/en/employment-social-development/services/foreign-workers/reports/2014/lmia-annual-statistics/agricultural.html#h2.5)
Page 1000 complete(https://www.canada.ca/en/employment-social-development/services/labour-standards/reports/wage-recovery-hearing.html)
Page 1500 complete(https://www.canada.ca/content/canadsite/en/employment-social-development/services/labour-relations/international.html)
Page 2000 complete(https://www.canada.ca/en/employment-social-development/services/indigenous/bulletins/spring-2013.html)
Page 2500 complete(https://www.canada.ca/en/employment-social-development/programs/disability/arc/federal-report2011/section2.html#fig2.2)
Page 3000 complete(https://www.canada.ca/en/employment-social-development/programs/ei/ei-list/ei-roe/user-requirements/appendix-a.html)
Page 3500 complete(https://www.canada.ca/en/employment-social-development/programs/ei/ei-list/reports/digest/chapter-23/compassionate-care-benefits.html#a23_2_4)
Page 4000

In [60]:
len(page_texts),len(visited),num_visited,len(queue)

(7379, 7379, 7379, 0)

In [61]:
import pandas as pd
df = pd.DataFrame({"url":visited,"text":page_texts})
if(URL_MUST_CONTAIN is not None):
    df.to_csv("data/crawl_data({} pages - {}).csv".format(num_visited,URL_MUST_CONTAIN),index=False)
else:
    df.to_csv("data/crawl_data({} pages).csv".format(num_visited),index=False)